# Import Database

In [3]:
import sqlite3
db = 'olist.db'
con = sqlite3.connect(db)

# create a cursor
cur = con.cursor()

# read the entire table name
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

# print table name
print(table_list)

[('olist_order_customer_dataset',), ('olist_order_dataset',), ('olist_order_reviews_dataset',), ('olist_order_payments_dataset',), ('olist_order_items_dataset',), ('olist_products_dataset',), ('olist_sellers_dataset',), ('olist_geolocation_dataset',), ('product_category_name_translation',)]


# Objective Analysis
- Most in-demand products
- Products that are least in demand
- Products with the best reviews
- Products with the worst reviews
- Total sales of products in each state

# Import Library & Table Needed in The Analysis

## Table used in this analysis :
- Orders table [olist_order_dataset]
- Items table [olist_order_items_dataset]
- Products table [olist_products_dataset]
- Translation table [product_category_name_translation]
- Customer table [olist_order_customer_dataset]
- Order reviews table [olist_order_reviews_dataset]

In [4]:
# Import library to be used in the analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Display the orders table
orders = pd.read_sql_query('SELECT * FROM olist_order_dataset', con)
orders.head()

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
# Display the order items table
order_items = pd.read_sql_query('SELECT * FROM olist_order_items_dataset', con)
order_items.head()

,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [6]:
# Display the products table
products = pd.read_sql_query('SELECT * FROM olist_products_dataset', con)
products.head()

,index,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [7]:
# Display the translation table
translation = pd.read_sql_query('SELECT * FROM product_category_name_translation', con)
translation.head()

,index,product_category_name,product_category_name_english
0,0,beleza_saude,health_beauty
1,1,informatica_acessorios,computers_accessories
2,2,automotivo,auto
3,3,cama_mesa_banho,bed_bath_table
4,4,moveis_decoracao,furniture_decor


In [8]:
# Display the reviews table
order_reviews = pd.read_sql_query('SELECT * FROM olist_order_reviews_dataset', con)
order_reviews.head()

,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [9]:
# Display the costumer table
customer = pd.read_sql_query('SELECT * FROM olist_order_customer_dataset', con)
customer.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


# Data Wrangling

In [10]:
# Merge all tables
tabel_gab1 = orders.iloc[:,[1,2,3,4]].merge(order_items.iloc[:,[1,3,6]], how="left", on = "order_id")
tabel_gab1.head()

,order_id,customer_id,order_status,order_purchase_timestamp,product_id,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90


In [11]:
tabel_gab2 = tabel_gab1.merge(order_reviews.iloc[:,[2,3]], how="left", on = "order_id")
tabel_gab2.head()

,order_id,customer_id,order_status,order_purchase_timestamp,product_id,price,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90,5.0


In [12]:
tabel_gab3 = tabel_gab2.merge(customer.iloc[:,[1,5]], how="left", on = "customer_id")
tabel_gab3.head()

,order_id,customer_id,order_status,order_purchase_timestamp,product_id,price,review_score,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,4.0,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70,4.0,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90,5.0,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,5.0,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90,5.0,SP


In [13]:
tabel_gab4 = tabel_gab3.merge(products.iloc[:,[1,2]], how="left", on = "product_id")
tabel_gab4.head()

,order_id,customer_id,order_status,order_purchase_timestamp,product_id,price,review_score,customer_state,product_category_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,4.0,SP,utilidades_domesticas
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70,4.0,BA,perfumaria
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90,5.0,GO,automotivo
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,5.0,RN,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90,5.0,SP,papelaria


In [14]:
tabel_gab5 = tabel_gab4.merge(translation.iloc[:,[1,2]], how="left", on = "product_category_name")
tabel_gab5.head()

,order_id,customer_id,order_status,order_purchase_timestamp,product_id,price,review_score,customer_state,product_category_name,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,4.0,SP,utilidades_domesticas,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70,4.0,BA,perfumaria,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90,5.0,GO,automotivo,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,5.0,RN,pet_shop,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90,5.0,SP,papelaria,stationery


In [15]:
tabel_gab_all = tabel_gab5.drop('product_category_name', axis = 1)
tabel_gab_all.head()

,order_id,customer_id,order_status,order_purchase_timestamp,product_id,price,review_score,customer_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,4.0,SP,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70,4.0,BA,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90,5.0,GO,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,5.0,RN,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90,5.0,SP,stationery


# Data Cleaning

In [16]:
# Check table info
tabel_gab_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114092 entries, 0 to 114091
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       114092 non-null  object 
 1   customer_id                    114092 non-null  object 
 2   order_status                   114092 non-null  object 
 3   order_purchase_timestamp       114092 non-null  object 
 4   product_id                     113314 non-null  object 
 5   price                          113314 non-null  float64
 6   review_score                   113131 non-null  float64
 7   customer_state                 114092 non-null  object 
 8   product_category_name_english  111678 non-null  object 
dtypes: float64(2), object(7)
memory usage: 8.7+ MB


In [17]:
# Check for missing values
tabel_gab_all.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
product_id                        778
price                             778
review_score                      961
customer_state                      0
product_category_name_english    2414
dtype: int64

In [18]:
# Set missing values in product_id and product_category_name_english columns to Unknown
tabel_gab_all.loc[tabel_gab_all["product_id"].isna(),"product_id"] = "Unknown"
tabel_gab_all.loc[tabel_gab_all["product_category_name_english"].isna(),"product_category_name_english"] = "Unknown"

# Replace missing values price and review score with average price and average review score
mean_price = tabel_gab_all['price'].mean()
tabel_gab_all['price'].fillna(mean_price, inplace=True)
mean_review_score = tabel_gab_all['review_score'].mean()
tabel_gab_all['review_score'].fillna(mean_review_score, inplace=True)
tabel_gab_all.isna().sum()


order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
product_id                       0
price                            0
review_score                     0
customer_state                   0
product_category_name_english    0
dtype: int64

In [19]:
# Check for duplicate data
tabel_gab_all.duplicated().sum()

10664

In [20]:
# Remove duplicates
tabel_gab_all = tabel_gab_all.drop_duplicates(keep='first')
tabel_gab_all.duplicated().sum()

0

In [23]:
# Cek order status
tabel_gab_all['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

It can be seen that there are unavailable and canceled statuses, which means that products with this status are not sold. Therefore, the status must be dropped so that the analysis results only focus on items that have been sold.

In [27]:
# Dropping unavailable and canceled status orders
tabel_gab_all = tabel_gab_all[~tabel_gab_all['order_status'].isin(["unavailable", "canceled"])]
tabel_gab_all['order_status'].unique()


array(['delivered', 'invoiced', 'shipped', 'processing', 'created',
       'approved'], dtype=object)

In [28]:
# Checking data info
tabel_gab_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102187 entries, 0 to 114091
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       102187 non-null  object 
 1   customer_id                    102187 non-null  object 
 2   order_status                   102187 non-null  object 
 3   order_purchase_timestamp       102187 non-null  object 
 4   product_id                     102187 non-null  object 
 5   price                          102187 non-null  float64
 6   review_score                   102187 non-null  float64
 7   customer_state                 102187 non-null  object 
 8   product_category_name_english  102187 non-null  object 
dtypes: float64(2), object(7)
memory usage: 7.8+ MB


Now we have 102187 data ready to be analyzed.

# Analysis & Visualizations

# Most in-demand products

In [87]:
most_product_best = tabel_gab_all[["product_category_name_english","order_id"]].groupby("product_category_name_english").count()
most_product_best = most_product_best.sort_values("order_id", ascending=False).reset_index()

most_product_best

,product_category_name_english,order_id
0,bed_bath_table,10203
1,health_beauty,9001
2,sports_leisure,7824
3,computers_accessories,6878
4,furniture_decor,6786
...,...,...
67,home_comfort_2,24
68,la_cuisine,13
69,cds_dvds_musicals,12
70,fashion_childrens_clothes,8


In [94]:
# Visualization of the 10 most in-demand products
top_10_product_best = most_product_best.head(10)
top_10_product_best = top_10_product_best.sort_values(by='order_id', ascending=True)

fig = px.bar(top_10_product_best, x='order_id', y='product_category_name_english',
             orientation='h', title='Top 10 Products by Order Quantity',
             labels={'order_id': 'Order Quantity', 'product_category_name_english': 'Product Kategory'})

fig.update_layout(template='plotly_dark')

fig.show()


# Products that are least in demand

In [89]:
most_product_worst = tabel_gab_all[["product_category_name_english","order_id"]].groupby("product_category_name_english").count()
most_product_worst = most_product_worst.sort_values("order_id", ascending=True).reset_index()

most_product_worst

,product_category_name_english,order_id
0,security_and_services,2
1,fashion_childrens_clothes,8
2,cds_dvds_musicals,12
3,la_cuisine,13
4,arts_and_craftmanship,24
...,...,...
67,furniture_decor,6786
68,computers_accessories,6878
69,sports_leisure,7824
70,health_beauty,9001


In [95]:
# Visualization of the 10 least desirable products
top_10_product_worst = most_product_worst.head(10)
top_10_product_worst = top_10_product_worst.sort_values(by='order_id', ascending=False)

fig = px.bar(top_10_product_worst, x='order_id', y='product_category_name_english',
             orientation='h', title='Top 10 Lowest Products by Order Quantity',
             labels={'order_id': 'Order Quantity', 'product_category_name_english': 'Product Kategory'})

fig.update_layout(template='plotly_dark')

fig.show()

# Products with the best reviews

In [97]:
product_review_score_best = tabel_gab_all[["product_category_name_english","review_score"]].groupby("product_category_name_english").mean()
product_review_score_best = product_review_score_best.sort_values("review_score", ascending=False).reset_index()

product_review_score_best

,product_category_name_english,review_score
0,cds_dvds_musicals,4.666667
1,fashion_childrens_clothes,4.500000
2,books_general_interest,4.481966
3,costruction_tools_tools,4.412897
4,food_drink,4.407725
...,...,...
67,furniture_mattress_and_upholstery,3.815789
68,fashio_female_clothing,3.682927
69,fashion_male_clothing,3.664074
70,office_furniture,3.597065


In [98]:
# Visualization of top 10 products with the best reviews
top_10_product_review_best = product_review_score_best.head(10)
top_10_product_review_best = top_10_product_review_best.sort_values(by = "review_score", ascending = True)

fig = go.Figure(data=[go.Bar(y=top_10_product_review_best['product_category_name_english'], x=top_10_product_review_best['review_score'], orientation='h')])
fig.update_layout(title='Top 10 Best Average Product Review Scores',
                  xaxis_title='Average Review Score',
                  yaxis_title='Product Category',
                  template='plotly_dark')

fig.show()


# Products with the worst reviews

In [92]:
product_review_score_worst = tabel_gab_all[["product_category_name_english","review_score"]].groupby("product_category_name_english").mean()
product_review_score_worst = product_review_score_worst.sort_values("review_score", ascending=True).reset_index()

product_review_score_worst

,product_category_name_english,review_score
0,security_and_services,2.500000
1,office_furniture,3.597065
2,fashion_male_clothing,3.664074
3,fashio_female_clothing,3.682927
4,furniture_mattress_and_upholstery,3.815789
...,...,...
67,food_drink,4.407725
68,costruction_tools_tools,4.412897
69,books_general_interest,4.481966
70,fashion_childrens_clothes,4.500000


In [99]:
# Visualization of the top 10 products with the worst reviews
top_10_product_review_worst = product_review_score_worst.sort_values(by = "review_score", ascending = False).tail(10)

fig = go.Figure(data=[go.Bar(y=top_10_product_review_worst['product_category_name_english'], x=top_10_product_review_worst['review_score'], orientation='h')])
fig.update_layout(title='Top 10 Worst Average Product Review Scores',
                  xaxis_title='Average Review Score',
                  yaxis_title='Product Category',
                  template='plotly_dark')

fig.show()

# Total sales of products in each state

In [84]:
most_product_per_province = tabel_gab_all[["product_category_name_english","order_id","customer_state"]].groupby(["product_category_name_english","customer_state"]).count()
most_product_per_province = most_product_per_province.sort_values("order_id", ascending=False).reset_index()

most_product_per_province = most_product_per_province[["customer_state", "order_id"]].groupby("customer_state").sum()
most_product_per_province = most_product_per_province.sort_values("order_id", ascending=False).reset_index()

most_product_per_province

,customer_state,order_id
0,SP,42883
1,RJ,13225
2,MG,11959
3,RS,5637
4,PR,5149
5,SC,3729
6,BA,3476
7,DF,2217
8,GO,2096
9,ES,2092


In [102]:
# Visualization of total product sales per state
import plotly.express as px

# Take only the top 10 customer states
top_10_states = most_product_per_province.head(10).sort_values(by = 'order_id', ascending = True)

# Change customer status code
state_mapping = {
    'SP': 'São Paulo',
    'RJ': 'Rio de Janeiro',
    'MG': 'Minas Gerais',
    'RS': 'Rio Grande do Sul',
    'PR': 'Paraná',
    'SC': 'Santa Catarina',
    'BA': 'Bahia',
    'DF': 'Distrito Federal',
    'GO': 'Goiás',
    'ES': 'Espírito Santo'
}

top_10_states['customer_state'] = top_10_states['customer_state'].map(state_mapping)

# Make a visualization
fig = px.bar(top_10_states, x='order_id', y='customer_state', orientation='h', title='Top 10 Customer States by Order Amount', labels={'order_id':'Order Quantity', 'customer_state':'Customer State'})
fig.update_layout(template='plotly_dark')
fig.show()



# Conclusion
Based on the results of the analysis that has been carried out, some conclusions and insights that can be drawn are as follows:

1. Most Popular Products: From the analysis of sales data, it can be seen that bed bath table, health beauty, and sport leisure products are the three most popular products for customers. This could be an opportunity to focus on marketing and increasing the stock of these products.

2. Products that are not in demand: On the other hand, security and services, fashion childrens clothes, and cds_dvds_musicals are the least desirable products. It may be necessary to further evaluate these products and look for strategies to increase customer interest in these products.

3. Products with the Best Review Score: Although cds_dvds_musicals, fashions childrens clothes, and books general interest products have the best review scores, it doesn't hurt to keep focusing on quality and customer service to maintain positive review scores.

4. Products with the Lowest Review Score: Security and services, office furniture, and fashion male clothing products have the lowest review scores. It is important to conduct an in-depth analysis of the factors that lead to low review scores and find ways to improve product and service quality.

5. State with the Highest Total Sales: The states of São Paulo, Rio de Janeiro, and Minas Gerais are the states with the most total product sales. This shows that the market in these three states is quite large and needs to be well taken care of to meet customer needs.

Insights from this analysis can help in making business decisions, such as focusing on products that customers are interested in, improving product and service quality, and optimizing marketing in provinces with high total sales. In addition, this analysis also provides an overview of customer consumption trends and patterns that can be used to direct further business strategies.